<a href="https://colab.research.google.com/github/dar-tau/nlp-experiments/blob/master/introbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Initialization

In [ ]:
!pip install transformers datasets
# !pip install simpletransformers

In [16]:
from transformers import BertTokenizerFast, BertForQuestionAnswering
import torch
import datasets
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

## Functions

In [ ]:
def preprocess_data(data):
  answers = tokenizer(pd.DataFrame(data['answers'])['text'].apply(lambda x: x[0]).values.tolist())
  inputs = tokenizer(data['question'], data['context'])
  return inputs, answers

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased', padding = True, truncate = True)
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased', return_dict=True)
squad = datasets.load_dataset("squad")
optimizer = AdamW(model.parameters(), lr=5e-5)
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_train_steps)
model.train();

## Main

In [ ]:
inputs, answers = preprocess_data(squad['train'])

In [ ]:
nSamples = len(inputs)
for e in range(10):
    for i in tqdm(range(nSamples)):
      model(inputs[i].ids, label = answers[i].ids)